In [ ]:
import sys
# setting path
sys.path.append('../')

import pandas as pd

In [ ]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
RESULTS_PATH = MAIN_PATH + "results/"

# CAMELYON

In [ ]:
path = RESULTS_PATH + "results_camelyon.csv"
df = pd.read_csv(path)
df = df.drop(columns=["camelyon_test_loss", "camelyon_test_loss_mse", "camelyon_test_loss_kl"])

df["Version"] = ["New" for i in range(len(df))]
sh_row = {"Version" : "Old", "patch_size" : 64, "pretraining_epochs" : 1, "n_samples" : "?", "lambda_val" : "$\\theta=0.8$", "sigmaRui_sq" : "0.05", "camelyon_test_mse_rec" : "0.0712", "camelyon_test_ssim_rec" : "?", "camelyon_test_ssim_rec" : "?"}
df = df.append(sh_row, ignore_index=True)
df.columns = ["Patch size", "Pretraining epochs", "Num. samples", "$\lambda$", "$\sigma_{Rui}^2$", "Reconstruction MSE", "Reconstruction PSNR", "Reconstruction SSIM", "Version"]
df = df[["Version"] + list(df.columns[:-1])]
df

In [ ]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Results on Camelyon test dataset.", label="tab:results_camelyon",
    )
print(s)
# Fix header

# WSSB

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
#df[[c for c in df.columns if "mse_rec" in c]]

df = df[df["pretraining_epochs"] == 1] 
df = df[[c for c in df.columns if "mse_rec" in c]]
df["wssb_mean_test_mse_rec"] = df.mean(axis=1)
df["Name"] = "BCDnet (new)"
df = df[["Name"] + list(df.columns[:-1])]

df = df.append({"Name" : "BCDnet (old)", "wssb_Lung_test_mse_rec" : 0.0066, "wssb_Breast_test_mse_rec" : 0.0172, "wssb_Colon_test_mse_rec" : 0.0092, "wssb_mean_test_mse_rec" : 0.0110}, ignore_index=True)
df = df.append({"Name" : "BKSVD", "wssb_Lung_test_mse_rec" : 0.0004, "wssb_Breast_test_mse_rec" : 0.0003, "wssb_Colon_test_mse_rec" : 0.0003, "wssb_mean_test_mse_rec" : 0.0003}, ignore_index=True)

In [ ]:
df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Breast", "Colon", "Mean"]
df = df[["Organ"] + list(df.columns[:-1])]
df

In [ ]:
styler = df.style
styler = styler.hide(level=0, axis=0)  
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns))]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Reconstruction MSE on WSSB dataset.", label="tab:mse_rec_wssb",
    )
print(s)

## Stain metrics

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == 1] 
df = df[[c for c in df.columns if "psnr_gt" in c]]
df["Name"] = ["BCDnet (new)" for i in range(len(df))]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", 
    "wssb_Lung_test_psnr_gt_h" : 27.06, "wssb_Lung_test_psnr_gt_e" : 25.69, 
    "wssb_Breast_test_psnr_gt_h" : 21.81, "wssb_Breast_test_psnr_gt_e" : 24.53, 
    "wssb_Colon_test_psnr_gt_h" : 20.32, "wssb_Colon_test_psnr_gt_e" : 21.70}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_psnr_gt_h" : 32.67, "wssb_Lung_test_psnr_gt_e" : 30.61, 
    "wssb_Breast_test_psnr_gt_h" : 32.20, "wssb_Breast_test_psnr_gt_e" : 29.43, 
    "wssb_Colon_test_psnr_gt_h" : 34.08, "wssb_Colon_test_psnr_gt_e" : 33.32}, ignore_index=True)

df["wssb_mean_test_psnr_gt_h"] = df[[c for c in df.columns if "h" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt_e"] = df[[c for c in df.columns if "e" in c]].mean(axis=1)
df["wssb_mean_test_psnr_gt"] = df[["wssb_mean_test_psnr_gt_h", "wssb_mean_test_psnr_gt_e"]].mean(axis=1)

df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

In [ ]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth PSNR on WSSB dataset.", label="tab:gt_psnr_wssb",
    )
print(s)

In [ ]:
path = RESULTS_PATH + "results_wssb.csv"
df = pd.read_csv(path)
df = df.drop(columns=[c for c in df.columns if "loss" in c])
df = df[df["pretraining_epochs"] == 1] 
df = df[[c for c in df.columns if "ssim_gt" in c]]
df["Name"] = ["BCDnet (new)" for i in range(len(df))]
df = df[["Name"] + list(df.columns[:-1])]
df = df.append({"Name" : "BCDnet (old)", 
    "wssb_Lung_test_ssim_gt_h" : 0.9518, "wssb_Lung_test_ssim_gt_e" : 0.8201, 
    "wssb_Breast_test_ssim_gt_h" : 0.8962, "wssb_Breast_test_ssim_gt_e" : 0.7321, 
    "wssb_Colon_test_ssim_gt_h" : 0.8969, "wssb_Colon_test_ssim_gt_e" : 0.8560}, ignore_index=True)
df = df.append({"Name" : "BKSVD", 
    "wssb_Lung_test_ssim_gt_h" : 0.9764, "wssb_Lung_test_ssim_gt_e" : 0.9461, 
    "wssb_Breast_test_ssim_gt_h" : 0.9801, "wssb_Breast_test_ssim_gt_e" : 0.9632, 
    "wssb_Colon_test_ssim_gt_h" : 0.9826, "wssb_Colon_test_ssim_gt_e" : 0.9646}, ignore_index=True)
df["wssb_mean_test_ssim_gt_h"] = df[[c for c in df.columns if "h" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt_e"] = df[[c for c in df.columns if "e" in c]].mean(axis=1)
df["wssb_mean_test_ssim_gt"] = df[["wssb_mean_test_ssim_gt_h", "wssb_mean_test_ssim_gt_e"]].mean(axis=1)

df = df.transpose()
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(inplace=True)
df.columns.name = None
df = df.drop(columns=["index"])
df["Organ"] = ["Lung", "Lung", "Breast", "Breast", "Colon", "Colon", "Mean", "Mean", "Mean"]
df["Stain"] = ["H", "E", "H", "E", "H", "E", "H", "E", "Mean"]

df.set_index(["Organ", "Stain"], inplace=True)
df

In [ ]:
styler = df.style
s = styler.to_latex(
    column_format=f"{ ''.join(['c' for i in range(len(df.columns)+2)]) }", clines="skip-last;data", 
    hrules = True, position_float="centering", multirow_align="c", sparse_index="True", 
    caption="Ground truth SSIM on WSSB dataset.", label="tab:gt_ssim_wssb",
    )
print(s)